In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import json
import requests
from pprint import pprint

In [2]:
data = pd.read_csv('./data.tsv',sep='\t',skipinitialspace=True)
data.head()

,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,1.4158,2.9711,10.7935,7.5279,2.3352,8.1042,2.3096,3.3367,11.8639,12.7142,...,171.764,1434.24,0.331511,-0.932553,0.285048,-0.1435,-0.833982,0.767568,0.463969,1.904800
1,0.6280,1.8616,10.1770,7.4684,2.1915,8.5945,0.1379,2.9661,11.5816,12.2487,...,185.824,1469.19,0.894066,-0.446796,0.058519,-0.4624,0.715252,0.999105,0.988844,0.689742
2,0.9648,1.8103,10.1682,5.9705,2.0629,6.5349,2.8694,3.1185,11.7464,12.2074,...,187.576,1540.76,0.999982,0.460716,0.997809,-0.4624,0.723031,0.992935,0.682903,0.749580
3,0.7119,1.6221,10.1487,6.8678,2.0694,6.8806,1.5791,2.3003,11.5545,12.0659,...,189.938,1498.29,0.998794,-0.862448,0.329694,-0.4624,0.891745,0.015078,0.997127,0.984439
4,0.3797,1.6852,10.9601,5.0035,3.1659,5.9471,0.0858,2.6402,11.7458,12.7041,...,181.275,1465.11,0.442893,-0.990703,-0.151400,0.2245,0.617880,-0.506397,-0.997502,0.376126


In [3]:
# Get features and labels
features = data.iloc[:, :18].values
labels = data.iloc[:, 18:26].values
# Normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(features)
labels_scaled = labels


In [4]:

# Split into train, validation, and test sets
train_size = int(len(features_scaled) * 0.7)
val_size = int(len(features_scaled) * 0.1)
test_size = len(features_scaled) - train_size - val_size

train_features = features_scaled[:train_size, :]
train_labels = labels_scaled[:train_size]

val_features = features_scaled[train_size:train_size+val_size, :]
val_labels = labels_scaled[train_size:train_size+val_size]

test_features = features_scaled[train_size+val_size:, :]
test_labels = labels_scaled[train_size+val_size:]


In [5]:

# Create LSTM dataset
def create_lstm_dataset(features, labels, time_steps=1):
    dataX, dataY = [], []
    for i in range(len(features) - time_steps):
        a = features[i:(i + time_steps), :]
        dataX.append(a)
        dataY.append(labels[i + time_steps:i + time_steps + 1])
    return np.array(dataX), np.array(dataY)

time_steps = 7
trainX, trainY = create_lstm_dataset(train_features, train_labels, time_steps)
valX, valY = create_lstm_dataset(val_features, val_labels, time_steps)
testX, testY = create_lstm_dataset(test_features, test_labels, time_steps)

In [6]:
# 构建LSTM模型
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(time_steps, 18)))
model.add(Dense(8))  # 8 output units for 8 labels

learning_rate = 0.001
optimizer = Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)


2023-08-21 07:56:09.248934: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-08-21 07:56:09.248976: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-21 07:56:09.248991: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (58c881efe57c): /proc/driver/nvidia/version does not exist
2023-08-21 07:56:09.249223: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.7/site-packages

In [ ]:

# 拟合模型
# Fit the model
history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=100, batch_size=32, verbose=1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']


Epoch 1/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.4397 - val_loss: 0.4294
Epoch 2/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.4382 - val_loss: 0.4310
Epoch 3/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.4380 - val_loss: 0.4302
Epoch 4/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.4379 - val_loss: 0.4299
Epoch 5/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.4379 - val_loss: 0.4300
Epoch 6/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.4378 - val_loss: 0.4310
Epoch 7/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.4379 - val_loss: 0.4300
Epoch 8/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.4377 - val_loss: 0.4294
Epoch 9/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.4377 - val_loss: 0.4308
Epoch 10/100
1094/1094 [==============================] - 6s 5ms/step - l

In [ ]:

# 绘制损失曲线
# Plot loss
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()


In [ ]:
# 预测测试集
# Predict on test set
test_predict = model.predict(testX)
testY=testY.reshape(len(testY),8)

In [ ]:
#精度评价
# Accuracy evaluation
for i in range(8):
    test_predict1=test_predict[:,i]
    testY1= testY[:,i]
    MSE = mean_squared_error(test_predict1, testY1)
    print("Mean Squared Error for Column", i + 1, ":", MSE)

In [ ]:
model_version = 3
tf.keras.models.save_model(
    model,
    f'/models/slot0/{model_version}/', # v1/models/slot0/为tensorflow-serving的模型根目录
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

In [ ]:
req_data = json.dumps({
            'inputs': test_features.values[:1].tolist()
        })  
print(req_data)
response = requests.post(f'http://fireeye-test-model-container:8501/v1/models/slot0/versions/{model_version}:predict', # 根据部署地址填写
                         data=req_data,
                         headers={"content-type": "application/json"})
if response.status_code != 200:
    raise RuntimeError('Request tf-serving failed: ' + response.text)
resp_data = json.loads(response.text)    
if 'outputs' not in resp_data \
                    or type(resp_data['outputs']) is not list:
    raise ValueError('Malformed tf-serving response')

print(resp_data)
print("{'outputs':",test_labels.values[:1].tolist())

print("y1 MSE:%.4f" % mean_squared_error(test_labels.values[:1].tolist(), resp_data['outputs']))